In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import datasets, linear_model
from sklearn import metrics
from statsmodels.api import OLS
import statsmodels.api as sm

from scipy import stats

In [ ]:
csv_path = r'C:\Users\The_Iron_Maiden\Documents\Python Scripts\College.csv'

college_data = pd.read_csv(csv_path)

# Boston

    CRIM - per capita crime rate by town
    ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
    INDUS - proportion of non-retail business acres per town.
    CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
    NOX - nitric oxides concentration (parts per 10 million)
    RM - average number of rooms per dwelling
    AGE - proportion of owner-occupied units built prior to 1940
    DIS - weighted distances to five Boston employment centres
    RAD - index of accessibility to radial highways
    TAX - full-value property-tax rate per $10,000
    PTRATIO - pupil-teacher ratio by town
    B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
    LSTAT - % lower status of the population
    MEDV - Median value of owner-occupied homes in $1000's

In [ ]:
csv_path = r'C:\Users\The_Iron_Maiden\Documents\Python Scripts\Boston.csv'

boston_data = pd.read_csv(csv_path)
boston_data.keys()

num_samples = boston_data.shape[0]

In [ ]:
boston_data.head(2)

__first problem wants us to build a model predicting median value from lstat; let's look at the relationship__

In [ ]:
# plot relationship btw lstat (% of population with lower status) and median home value (in 1000's)
ax = sns.scatterplot(x="lstat", y="medv", data=boston_data)

### Use statsmodel to get a summary of a fitted linear model

https://stats.stackexchange.com/questions/146804/difference-between-statsmodel-ols-and-scikit-linear-regression

scikit learn is geared towards predictive modeling, so doesn't have an R summary equivalent for LR

_"Linear regression is in its basic form the same in statsmodels and in scikit-learn."_

_"Statsmodels follows largely the traditional model where we want to know __how well a given model fits the data, and what variables "explain" or affect the outcome, or what the size of the effect is__. Scikit-learn follows the machine learning tradition where the main supported task is __chosing the "best" model for prediction__."_


In [ ]:
# Grab the dependent and independent variables
simple_LR_y = pd.DataFrame(boston_data['medv'])
simple_LR_x = pd.DataFrame(boston_data['lstat'])

In [ ]:
sm_X = sm.add_constant(simple_LR_x) # THIS IS VERY IMPORTANT - ADDS CONSTANT for INTERCEPT

OLS(simple_LR_y,sm_X).fit().summary()

### Use scikit learn linear regression to make a model

In [ ]:
# initialize linear regression object
regr = linear_model.LinearRegression()
# fit the LR
regr.fit(simple_LR_x, simple_LR_y)


In [ ]:
# https://towardsdatascience.com/a-beginners-guide-to-linear-regression-in-python-with-scikit-learn-83a8f7ae2b4f

print('Intercept: \n', regr.intercept_[0])

# The coefficients
print('Coefficients: \n', regr.coef_[0])

In [ ]:
# make a vector of x values that span the limits of the dataset
padding = 3
x_data_min = simple_LR_x.min()['lstat']
x_data_max = simple_LR_x.max()['lstat']
x_line = np.arange(x_data_min-padding,x_data_max+padding)
# predict y values for these x values
y_line_pred = regr.predict(x_line.reshape(-1, 1))
# we'll use these x and y values to plot a line representing the best fit regression line from our model

ax = sns.scatterplot(x="lstat", y="medv", data=boston_data)
plt.plot(x_line, y_line_pred, color='r')
plt.show()

### Some insight on the model: 

the lower lstat datapoints introduce nonlinearity in the data. They are harder to predict and also overestimates predictions across lstat values

In [ ]:
# use .predict() to calculate the model's medv estimates for each data point's lstat
y_pred = regr.predict(simple_LR_x)

print('Mean Absolute Error:', metrics.mean_absolute_error(simple_LR_y, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(simple_LR_y, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(simple_LR_y, y_pred)))
print('R^2:', metrics.r2_score(simple_LR_y, y_pred))

In [ ]:
# calculate residual sum of squares (RSS)

rss = ( (simple_LR_y - y_pred)**2 ).sum(axis = 0)
print('RSS: ' + str(rss))

print('')

rse = ( (1/(num_samples-2)) * rss )**0.5
print('RSE: ' + str(rse))

print('')

tss = np.sum( (simple_LR_y - np.mean(simple_LR_y))**2 )
r_squared = 1 - (rss/tss)
print('R^2: ' + str(r_squared))

# Multivariable

In [ ]:
boston_data.head(2)

In [ ]:
# Grab the dependent and independent variables
multiple_LR_y = pd.DataFrame(boston_data['medv'])
multiple_LR_x = pd.DataFrame(boston_data.drop(['Unnamed: 0','medv'] , axis='columns')) # get rid of irrelevant and outcome columns

In [ ]:
sm_X_mult = sm.add_constant(multiple_LR_x) # THIS IS VERY IMPORTANT - ADDS CONSTANT for INTERCEPT

OLS(multiple_LR_y,sm_X_mult).fit().summary()

# we get similar coefficient as R's book

__NOX - nitric oxides concentration (parts per 10 million)__

__lstat value changed - potentially b/c of correlations with other variables__

### interaction terms

Below, we'll use stats model formula to define the dependent, independent variables, interactions, and any transformations. This roughly replicates how one would write the model formulas in R.

The language we use is called Patsy which takes the form of a string that is interpretted and converted to an equation that statsmodels uses to generate the linear model

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
smf.ols(formula='medv ~ lstat * age', data=boston_data).fit().summary()

__The above warning indicates collinearity between variables; let's see how correlated lstat and age are:__

In [ ]:
pear_r = stats.pearsonr(boston_data['lstat'], boston_data['age'])

sns.scatterplot(boston_data['lstat'], boston_data['age']).set_title("Pearson R = " + str(pear_r[0]))

last thing to do is __apply nonlinear functions to regression variables__

In [ ]:
# the I() evaluates its contents and sends that to python as a single variable

# https://stackoverflow.com/questions/16665833/statsmodels-specifying-non-linear-regression-models-using-patsy
# https://patsy.readthedocs.io/en/latest/builtins-reference.html

smf.ols(formula='medv ~ lstat + I(lstat**2)', data=boston_data).fit().summary()